Instalar librerias

Conectar al dataset y descargar spider

In [4]:
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Probar los modelos

In [8]:
import json
import torch
import random
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Rutas de los modelos y dataset
MODEL_PATH = "/content/drive/MyDrive/spider_models_fine_Tuned/T5-SQL"
SPIDER_DATASET_PATH = "/content/drive/MyDrive/spider/test.json"

In [10]:
# Cargar modelo y tokenizer
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)
tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model.eval()

def generate_sql(question, db_id, schema):
    """Genera SQL a partir de una pregunta en lenguaje natural con información del esquema."""
    formatted_input = f"Database: {db_id} | Schema: {schema} | Question: {question}"
    inputs = tokenizer(formatted_input, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=5,  # Evita exploraciones erróneas
            early_stopping=True,
            repetition_penalty=1.8,  # Ajustado para menor penalización
            length_penalty=2.0,  # Ajustado para fomentar estructuras más completas
            no_repeat_ngram_size=4,
            do_sample=False,  # Evita respuestas erráticas
            temperature=0.3  # Reducida para más determinismo
        )
    generated_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Verificar si la salida es válida
    valid_keywords = ["SELECT", "FROM", "WHERE", "GROUP BY", "ORDER BY", "JOIN", "LIMIT"]
    if not any(keyword in generated_sql.upper() for keyword in valid_keywords):
        generated_sql = "INVALID SQL"

    return generated_sql

# Cargar el dataset de Spider
with open(SPIDER_DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# Obtener el esquema de la base de datos real
def get_schema(db_id):
    """Extraer esquema real de la base de datos."""
    schema_dict = {
        "db1": "Tables: Rooms, Buildings | Columns: room_size, building_address",
        "db2": "Tables: Customers, Orders | Columns: customer_name, order_quantity, invoice_status",
        "db3": "Tables: Conference, Participants | Columns: conference_id, conference_name, year"
    }
    return schema_dict.get(db_id, "Schema not available")

# Seleccionar 5 preguntas aleatorias
random_samples = random.sample(data, 5)

In [13]:


# Evaluar para las preguntas aleatorias
results = []
for sample in random_samples:
    question = sample["question"]
    db_id = sample["db_id"]
    expected_sql = sample["query"]
    schema = get_schema(db_id)  # Obtener esquema de la base de datos
    generated_sql = generate_sql(question, db_id, schema)

    accuracy = 1 if generated_sql.strip().lower() == expected_sql.strip().lower() else 0
    results.append({
        "question": question,
        "expected_sql": expected_sql,
        "generated_sql": generated_sql,
        "accuracy": accuracy
    })

# Mostrar los resultados
for result in results:
    print("Pregunta:", result["question"])
    print("SQL Esperado:", result["expected_sql"])
    print("SQL Generado:", result["generated_sql"])
    print("Accuracy:", result["accuracy"])
    print("-" * 50)

# Calcular precisión global
overall_accuracy = sum(r["accuracy"] for r in results) / len(results)
print(f"Accuracy general: {overall_accuracy * 100:.2f}%")


Pregunta: Show name, type of powertrain, and annual fuel cost for all vehicles with model year 2013 or 2014.
SQL Esperado: SELECT name ,  type_of_powertrain ,  annual_fuel_cost FROM vehicles WHERE model_year  =  2013 OR model_year  =  2014
SQL Generado: SELECT name, type_of_powertrain, annual_fuel_cost FROM vehicles AS T1 JOIN model_years AS T2 ON T1.model_id = T2.model_year = "2013" OR T2.model-year = 2014
Accuracy: 0
--------------------------------------------------
Pregunta: Find the distinct titles of all the paintings that have a longer height than some painting on canvas?
SQL Esperado: SELECT DISTINCT title FROM paintings WHERE height_mm  >  (SELECT min(height_mm) FROM paintings WHERE mediumOn  =  "canvas")
SQL Generado: SELECT DISTINCT T1.title FROM paintings AS T1 JOIN painting AS T2 ON T1.artist_id = T2.artist_Id ORDER BY height > (SELECT HAVING height FROM canvas)
Accuracy: 0
--------------------------------------------------
Pregunta: What are the distinct descriptions of a